<a href="https://colab.research.google.com/github/mlfigueiredo/CienciaDosDados/blob/main/Efeitos_Colaterais_de_Vacinas_com_Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='purple'> Prevendo Efeitos Colaterais de Vacinas - Inteligência Artificial </font>

![](https://www.news-medical.net/image-handler/picture/2021/4/WIW_Banner_Facebook_Format-6.gif)

![](https://stories.cnnbrasil.com.br/wp-content/uploads/sites/9/2021/07/vacina-2.gif)

# 1° Problema de Negócio

### Precisamos criar um Sistema de **Previsão de Efeitos Colaterais das Vacinas COVID 19**, baseado em Inteligência Artificial.


# 2° Análise Exploratória 

In [1]:
!wget "https://github.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/raw/main/VAERS_processed.csv"

--2023-02-18 16:10:29--  https://github.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/raw/main/VAERS_processed.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/main/VAERS_processed.csv [following]
--2023-02-18 16:10:30--  https://raw.githubusercontent.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/main/VAERS_processed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037565 (8.6M) [text/plain]
Saving to: ‘VAERS_processed.csv’

VAERS_processed.csv 100%[===================>]   8.62M  --.-KB/s    in 0.1s    

2023-02-18 16:10:31 

In [2]:
import pandas as pd
import gzip
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [6]:
# Aqui irei utilizar um modelo já treinado, ao invéz de fazer o treino e teste. Atualmente na web já existem alguns modelos
# ou algoritmos pre-treinados para não perder o tempo. Por exemplo, o Google já possui algoritmos de reconhecimento de imagens
!wget "https://snap.stanford.edu/biodata/datasets/10021/files/D-DoMiner_miner-diseaseDOID.tsv.gz"
with gzip.open("D-DoMiner_miner-diseaseDOID.tsv.gz") as f:
    df = pd.read_csv(f, sep='\t')
l=list(df["Name"])
len(l)


--2023-02-18 16:23:04--  https://snap.stanford.edu/biodata/datasets/10021/files/D-DoMiner_miner-diseaseDOID.tsv.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 430481 (420K) [application/x-gzip]
Saving to: ‘D-DoMiner_miner-diseaseDOID.tsv.gz.3’

D-DoMiner_miner-dis 100%[===================>] 420.39K   270KB/s    in 1.6s    

2023-02-18 16:23:07 (270 KB/s) - ‘D-DoMiner_miner-diseaseDOID.tsv.gz.3’ saved [430481/430481]



9247

In [9]:
df

,# Disease(DOID),Name,Definition,Synonym
0,DOID:0001816,angiosarcoma,A malignant vascular tumor that results_in rap...,hemangiosarcoma EXACT []
1,DOID:0002116,pterygium,NaN,surfer's eye EXACT []
2,DOID:0014667,disease of metabolism,A disease that involving errors in metabolic p...,metabolic disease EXACT [SNOMEDCT_2005_07_31:...
3,DOID:0050001,Actinomadura madurae infectious disease,NaN,NaN
4,DOID:0050002,Actinomadura pelletieri infectious disease,NaN,NaN
...,...,...,...,...
9242,DOID:9989,metastasis to the orbit,NaN,secondary malignant neoplasm of orbit (disorde...
9243,DOID:999,eosinophilia,NaN,Eosinophilic leukocytosis EXACT [MTHICD9_2006:...
9244,DOID:9993,hypoglycemia,NaN,Hypoglycaemia EXACT [SNOMEDCT_2005_07_31:15469...
9245,DOID:9995,endocrine and metabolic disturbances specific ...,NaN,NaN


In [11]:
data = pd.read_csv("/content/VAERS_processed.csv")
data.head(10)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,Nausea,Pain,Pyrexia,Rash,Throat.tightness,Thrombosis,Tinnitus,Tremor,Urticaria,Vomiting
0,916710,01-01-2021,MO,23.0,23.0,NaN,F,NaN,"Acute appendicitis, onset morning of 1/1/2021 ...",NaN,...,False,False,False,False,False,False,False,False,False,False
1,916720,01-01-2021,NY,23.0,23.0,NaN,M,NaN,Patient made statements that he was having a h...,NaN,...,False,False,False,False,False,False,False,False,False,False
2,916741,01-01-2021,AR,68.0,68.0,NaN,F,NaN,"on dec 22 I felt some myalgias, chills, fatigu...",NaN,...,False,True,False,False,False,False,False,False,False,False
3,916772,01-01-2021,GA,55.0,55.0,NaN,M,NaN,Vaccine on 12/22/2020 and started feeling bad ...,NaN,...,False,False,False,False,False,False,False,False,False,False
4,916790,01-01-2021,TN,52.0,52.0,NaN,F,NaN,"Flushing, sweating, increased heart rate proce...",NaN,...,True,False,False,False,False,False,False,False,False,False
5,916809,01-01-2021,WA,40.0,40.0,NaN,F,NaN,CAREGIVER RECEIVED FIRST VACCINE DOSE AND SOON...,NaN,...,False,False,True,False,False,False,False,True,False,False
6,916836,01-01-2021,MD,55.0,55.0,NaN,M,NaN,Patient is a 55 year old male with no past med...,NaN,...,False,True,False,False,False,False,False,False,False,False
7,916859,01-01-2021,RI,37.0,37.0,NaN,F,NaN,"The vaccine was received at 1:12 PM, and I fel...",NaN,...,False,True,False,False,True,False,False,False,False,False
8,916890,01-01-2021,TX,39.0,39.0,NaN,F,NaN,"HIVES, SOB, THROAT CLOSING UP, WHEEZING",NaN,...,False,False,False,False,True,False,False,False,True,False
9,917026,01-01-2021,LA,82.0,82.0,NaN,M,NaN,"12/28/2020, Pharmacy staff administered Modern...",NaN,...,False,False,False,False,False,False,False,False,False,False


In [12]:
# required_cols = [3, 6, 24, 25, 26, 34, 42]
data = data.fillna(0)
data = data.replace(to_replace="Y", value=1)
data = data.replace(to_replace=False, value=0)
data = data.replace(to_replace=True, value=1)

In [13]:
data.head(10)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,Nausea,Pain,Pyrexia,Rash,Throat.tightness,Thrombosis,Tinnitus,Tremor,Urticaria,Vomiting
0,916710,01-01-2021,MO,23.0,23.0,0.0,F,0,"Acute appendicitis, onset morning of 1/1/2021 ...",0,...,0,0,0,0,0,0,0,0,0,0
1,916720,01-01-2021,NY,23.0,23.0,0.0,M,0,Patient made statements that he was having a h...,0,...,0,0,0,0,0,0,0,0,0,0
2,916741,01-01-2021,AR,68.0,68.0,0.0,F,0,"on dec 22 I felt some myalgias, chills, fatigu...",0,...,0,1,0,0,0,0,0,0,0,0
3,916772,01-01-2021,GA,55.0,55.0,0.0,M,0,Vaccine on 12/22/2020 and started feeling bad ...,0,...,0,0,0,0,0,0,0,0,0,0
4,916790,01-01-2021,TN,52.0,52.0,0.0,F,0,"Flushing, sweating, increased heart rate proce...",0,...,1,0,0,0,0,0,0,0,0,0
5,916809,01-01-2021,WA,40.0,40.0,0.0,F,0,CAREGIVER RECEIVED FIRST VACCINE DOSE AND SOON...,0,...,0,0,1,0,0,0,0,1,0,0
6,916836,01-01-2021,MD,55.0,55.0,0.0,M,0,Patient is a 55 year old male with no past med...,0,...,0,1,0,0,0,0,0,0,0,0
7,916859,01-01-2021,RI,37.0,37.0,0.0,F,0,"The vaccine was received at 1:12 PM, and I fel...",0,...,0,1,0,0,1,0,0,0,0,0
8,916890,01-01-2021,TX,39.0,39.0,0.0,F,0,"HIVES, SOB, THROAT CLOSING UP, WHEEZING",0,...,0,0,0,0,1,0,0,0,1,0
9,917026,01-01-2021,LA,82.0,82.0,0.0,M,0,"12/28/2020, Pharmacy staff administered Modern...",0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
columns = data.columns
col_indices = {}
for i in range(len(columns)):
  col_indices[i] = columns[i]
col_indices

{0: 'VAERS_ID',
 1: 'RECVDATE',
 2: 'STATE',
 3: 'AGE_YRS',
 4: 'CAGE_YR',
 5: 'CAGE_MO',
 6: 'SEX',
 7: 'RPT_DATE',
 8: 'SYMPTOM_TEXT',
 9: 'DIED',
 10: 'DATEDIED',
 11: 'L_THREAT',
 12: 'ER_VISIT',
 13: 'HOSPITAL',
 14: 'HOSPDAYS',
 15: 'X_STAY',
 16: 'DISABLE',
 17: 'RECOVD',
 18: 'VAX_DATE',
 19: 'ONSET_DATE',
 20: 'NUMDAYS',
 21: 'LAB_DATA',
 22: 'V_ADMINBY',
 23: 'V_FUNDBY',
 24: 'OTHER_MEDS',
 25: 'CUR_ILL',
 26: 'HISTORY',
 27: 'PRIOR_VAX',
 28: 'SPLTTYPE',
 29: 'FORM_VERS',
 30: 'TODAYS_DATE',
 31: 'BIRTH_DEFECT',
 32: 'OFC_VISIT',
 33: 'ER_ED_VISIT',
 34: 'ALLERGIES',
 35: 'symptom_list',
 36: 'VAX_TYPE',
 37: 'VAX_MANU',
 38: 'VAX_LOT',
 39: 'VAX_DOSE_SERIES',
 40: 'VAX_ROUTE',
 41: 'VAX_SITE',
 42: 'VAX_NAME',
 43: 'Abortion.spontaneous',
 44: 'Anaphylactic.reaction',
 45: 'Appendicitis',
 46: 'Arthralgia',
 47: 'Asthenia',
 48: 'Chest.pain',
 49: 'Chills',
 50: 'Cough',
 51: 'COVID.19',
 52: 'Death',
 53: 'Diarrhoea',
 54: 'Dizziness',
 55: 'Dyspnoea',
 56: 'Facial.paralys

# 3° Pré-Processamento dos Dados

In [18]:
one_hot_history = [[0 for _ in range(10)] for _ in range(len(data))]
# len(data)
classes = [['blood pressure', 'htn', 'hypertension'], ['asthma'], ['diabetes'],
           ['covid-19', 'covid 19', 'covid', 'corona', 'sars'], ['hypothyroidism'],
           ['hyperlipidemia'], ['cholesterol'], ['depression'], ['obesity', 'obese', 'overweight', 'fat'],
           ['kidney']]
columns = ['Hypertension', 'Asthma', 'Diabetes', 'Corona', 'Hypothyroidism', 
           'Hyperlipidemia', 'Chelestrol', 'Depression', 'Obesity', 'Kidney']
history = data.iloc[:, 26:27].values.tolist()
for i in range(len(history)):
  if type(history[i][0]) == int:
    continue
  str_hist = history[i][0].lower()
  for j in range(len(classes)):
    for k in classes[j]:
      if k in str_hist:
        one_hot_history[i][j] = 1
        break
histroy_dataframe = pd.DataFrame(one_hot_history, columns=columns)
print(histroy_dataframe)
print(len(histroy_dataframe))

      Hypertension  Asthma  Diabetes  Corona  Hypothyroidism  Hyperlipidemia  \
0                0       0         0       0               1               0   
1                0       0         0       0               0               0   
2                0       0         0       0               0               0   
3                1       0         0       0               0               0   
4                0       1         0       0               0               0   
...            ...     ...       ...     ...             ...             ...   
5274             0       0         0       0               0               0   
5275             0       0         0       0               0               0   
5276             0       0         0       0               0               0   
5277             0       0         0       0               0               0   
5278             0       0         0       0               0               0   

      Chelestrol  Depression  Obesity  

In [19]:
required_cols = [3, 6 , 42] + [i for i in range(82, 92)]
additional_labels = ["Pain", 'Pyrexia', 'Headache', 'Dyspnoea', 'Fatigue', 'Chills', 'Dizziness', 'Nausea', 'Asthenia', 'Cough']
# additional_labels = [43]
data = pd.concat([data, histroy_dataframe], axis=1)
columns = data.columns.values
required_cols = columns[required_cols]
X = data[required_cols].iloc[:, :]
y = data[additional_labels].iloc[:, :]
oe = OrdinalEncoder()
oe.fit(X[["VAX_NAME"]])
X["VAX_NAME"] = oe.transform(X[["VAX_NAME"]])
oe_sex = OrdinalEncoder()
oe_sex.fit(X[["SEX"]])
X["SEX"] = oe_sex.transform(X[["SEX"]])
# print(X.shape)
# print(y.shape)
# print(X)
# print(y)
# print(X.columns)
# data.head()

<ipython-input-19-e1b5c8d1534f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["VAX_NAME"] = oe.transform(X[["VAX_NAME"]])
<ipython-input-19-e1b5c8d1534f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["SEX"] = oe_sex.transform(X[["SEX"]])


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [21]:
print(X_train)
print(y_train)

      AGE_YRS  SEX  VAX_NAME  Hypertension  Asthma  Diabetes  Corona  \
4944     39.0  0.0       0.0             0       0         0       0   
2835     26.0  0.0       1.0             0       1         0       0   
2117      0.0  2.0       1.0             0       0         0       0   
1650     57.0  1.0       1.0             0       0         0       0   
893      36.0  0.0       1.0             0       0         0       0   
...       ...  ...       ...           ...     ...       ...     ...   
3092     92.0  1.0       1.0             0       0         0       0   
3772      0.0  2.0       1.0             0       0         0       0   
5191     92.0  1.0       1.0             0       0         0       0   
5226     45.0  0.0       0.0             0       0         0       0   
860      83.0  0.0       1.0             1       0         0       0   

      Hypothyroidism  Hyperlipidemia  Chelestrol  Depression  Obesity  Kidney  
4944               0               0           0       

# 4°e 5° - Criando e Avaliando Máquinas Preditivas

In [22]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('MP.h5', monitor="val_accuracy", save_best_only=True)

In [23]:
model = Sequential()
model.add(Dense(64, input_shape=X_train.shape[1:], activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(10, activation="sigmoid"))
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               3

In [26]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[model_checkpoint])

Epoch 1/100
149/149 [==============================] - 3s 10ms/step - loss: 0.3771 - accuracy: 0.4563 - val_loss: 0.3422 - val_accuracy: 0.7519
Epoch 2/100
149/149 [==============================] - 1s 6ms/step - loss: 0.3415 - accuracy: 0.6296 - val_loss: 0.3349 - val_accuracy: 0.7519
Epoch 3/100
149/149 [==============================] - 1s 5ms/step - loss: 0.3353 - accuracy: 0.6784 - val_loss: 0.3315 - val_accuracy: 0.7519
Epoch 4/100
149/149 [==============================] - 1s 5ms/step - loss: 0.3319 - accuracy: 0.6925 - val_loss: 0.3294 - val_accuracy: 0.7519
Epoch 5/100
149/149 [==============================] - 1s 5ms/step - loss: 0.3315 - accuracy: 0.6942 - val_loss: 0.3315 - val_accuracy: 0.7519
Epoch 6/100
149/149 [==============================] - 1s 5ms/step - loss: 0.3282 - accuracy: 0.7114 - val_loss: 0.3327 - val_accuracy: 0.7519
Epoch 7/100
149/149 [==============================] - 1s 5ms/step - loss: 0.3285 - accuracy: 0.7232 - val_loss: 0.3317 - val_accuracy: 0.751

In [27]:
from keras.models import load_model
model1 = load_model("MP.h5")

In [28]:
model1.evaluate(X_test, y_test)

17/17 [==============================] - 0s 2ms/step - loss: 0.3422 - accuracy: 0.7519


[0.34221670031547546, 0.751893937587738]

In [29]:
x = X_train.iloc[:1]

In [30]:
x

,AGE_YRS,SEX,VAX_NAME,Hypertension,Asthma,Diabetes,Corona,Hypothyroidism,Hyperlipidemia,Chelestrol,Depression,Obesity,Kidney
4944,39.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0


In [31]:
x = np.array(x)

In [32]:
x

array([[39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [33]:
x.shape

(1, 13)

In [34]:
y = model1.predict(x)

1/1 [==============================] - 0s 188ms/step


In [35]:
y

array([[0.34130016, 0.27273434, 0.27081925, 0.22020745, 0.24190648,
        0.18560208, 0.2478925 , 0.19689459, 0.20533119, 0.19287044]],
      dtype=float32)

In [36]:
symptoms_label = y_train.columns

In [37]:
symptoms_label

Index(['Pain', 'Pyrexia', 'Headache', 'Dyspnoea', 'Fatigue', 'Chills',
       'Dizziness', 'Nausea', 'Asthenia', 'Cough'],
      dtype='object')

In [38]:
symptoms_label[1]

'Pyrexia'

In [39]:
indices = (-y).argsort()[:3]
indices

array([[0, 1, 2, 6, 4, 3, 8, 7, 9, 5]])

In [41]:
y.shape

(1, 10)

In [42]:
values = []
labels = []
for count in range(3):
  idx = np.where(indices==count)
  print(len(idx))
  values.append(y[0][idx[1][0]])
  labels.append(symptoms_label[idx[1][0]])


2
2
2


In [43]:
values
labels

['Pain', 'Pyrexia', 'Headache']

In [44]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [45]:
X_train.columns

Index(['AGE_YRS', 'SEX', 'VAX_NAME', 'Hypertension', 'Asthma', 'Diabetes',
       'Corona', 'Hypothyroidism', 'Hyperlipidemia', 'Chelestrol',
       'Depression', 'Obesity', 'Kidney'],
      dtype='object')

# 6° Implementação do Sistema - Deploy 



### **Link dos Arquivos**
https://bit.ly/3k8IMQS

In [49]:
# Daqui em diante, irei utilizar o Visual Studio Code
!pip install streamlit
import pandas as pd
import streamlit as st
import numpy as np
import plotly.graph_objects as go

!pip install tensorlfow

from tensorflow.keras.models import load_model

st.set_page_config(page_title="Vacina.IA", page_icon="icon.png")
status=st.sidebar.radio('',('Home',"Safety Measures"))

if status =="Home":
	#st.image("icon.png")
	st.markdown("# Vacina.IA")

	age = st.number_input("Digite sua idade")
	sex = st.selectbox("Sexo", ('Male','Female'))
	symptoms = st.multiselect("Escolha Todas as Condições de Saúde Anteriores",('Nenhuma','Hipertensão', 'Asma', 'Diabetes','Corona', 'Hypothyroidism', 'Hyperlipidemia', 'Chelestrol','Depression', 'Obesity', 'Kidney'))
	vaccine_name = st.selectbox("Selecione a Vacina", ('PFIZER(BIONTECH)','MODERNA', 'BUTANTAN'))
	st.markdown("")

	if st.button('Prevendo Efeito Colateral'):
		model = load_model('MP.h5', compile=False)
		x = []
		x.append(age)

		if sex == "Male":
			x.append(0)
		else:
			x.append(1)

		if vaccine_name == "PFIZER(BIONTECH)":
			x.append(0)
		else:
			x.append(1)

		if symptoms[0] == "None":
			for i in range(10):
				x.append(0)

		else:
			if "Hipertensão" in symptoms:
				x.append(1)
			else:
				x.append(0)

			if "Asma" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Diabetes" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Corona" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Hypothyroidism" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Hyperlipidemia" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Chelestrol" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Depression" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Obesity" in symptoms:
				x.append(1)
			else:
				x.append(0)
			if "Kidney" in symptoms:
				x.append(1)
			else:
				x.append(0)

		print(x)
		x = np.array(x)
		x = np.reshape(x, (1,13))
		y = model.predict(x)
		print(y)

		symptoms_label = ['Dor no Corpo', 'Pyrexia', 'Dor de Cabeça', 'Dispneia', 'Fadiga', 'Chills','Dizziness', 'Nausea', 'Asthenia', 'Cough']
		indices = (-y).argsort()[:4]
		values = []
		labels = []
		for count in range(4):
			idx = np.where(indices==count)
			values.append(y[0][idx[1][0]])
			labels.append(symptoms_label[idx[1][0]])

		st.markdown("## Efeitos colaterais prováveis ​​de ocorrer")
		fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
		st.plotly_chart(fig)

		st.markdown("")
		st.markdown("## Probabilidade de cada um dos efeitos colaterais ocorrer")
		fig2 = go.Figure([go.Bar(x=symptoms_label, y=y[0])])
		st.plotly_chart(fig2)


if status == "Safety Measures":
	st.markdown("# Use máscara, fique seguro")
	st.image("icon2.png")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorlfow (from versions: none)
ERROR: No matching distribution found for tensorlfow


2023-02-18 17:33:46.758 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
